<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, clear the conflict by installing a modern, unified set
!pip install --upgrade pip
!pip install numpy>=2.0.0 pandas>=2.2.2 fsspec==2025.3.0 s3fs==2025.3.0 xarray h5netcdf --quiet

# IMPORTANT: You must click 'Restart Session' in the prompt that appears
# or go to Runtime -> Restart session before running the next cell.

In [43]:
import xarray as xr
import s3fs
import pandas as pd
import numpy as np

# 1. Connect to S3
fs = s3fs.S3FileSystem(anon=True)

# 2. Define the exact verified 2024 paths
month = "202401"
an_path = f"nsf-ncar-era5/e5.oper.an.sfc/{month}/"
# Corrected fc_path for GHI (accumulated forecast)
fc_path = f"nsf-ncar-era5/e5.oper.fc.sfc.accumu/{month}/"

# Mapping the filenames, with corrected GHI paths for two files
files = {
    'temp': f"{an_path}e5.oper.an.sfc.128_167_2t.ll025sc.2024010100_2024013123.nc",
    'u_wind': f"{an_path}e5.oper.an.sfc.128_165_10u.ll025sc.2024010100_2024013123.nc",
    'v_wind': f"{an_path}e5.oper.an.sfc.128_166_10v.ll025sc.2024010100_2024013123.nc",
    'ghi_part1': f"{fc_path}e5.oper.fc.sfc.accumu.128_169_ssrd.ll025sc.2024010106_2024011606.nc", # GHI file for first half of January
    'ghi_part2': f"{fc_path}e5.oper.fc.sfc.accumu.128_169_ssrd.ll025sc.2024011606_2024020106.nc"  # GHI file for second half of January
}

def extract_all_vars(lat, lon, city_name):
    # Longitude conversion for ERA5 (0-360)
    adj_lon = lon if lon >= 0 else 360 + lon

    try:
        # Open datasets using the 'h5netcdf' engine for S3 speed
        ds_t = xr.open_dataset(fs.open(files['temp']), engine='h5netcdf', chunks={})
        ds_u = xr.open_dataset(fs.open(files['u_wind']), engine='h5netcdf', chunks={})
        ds_v = xr.open_dataset(fs.open(files['v_wind']), engine='h5netcdf', chunks={})

        # --- GHI Data Processing ---
        # Open both GHI parts
        ds_g1_raw = xr.open_dataset(fs.open(files['ghi_part1']), engine='h5netcdf', chunks={})
        ds_g2_raw = xr.open_dataset(fs.open(files['ghi_part2']), engine='h5netcdf', chunks={})

        # Select the first (and usually only) 'forecast_initial_time' entry
        ds_g1 = ds_g1_raw.isel(forecast_initial_time=0)
        ds_g2 = ds_g2_raw.isel(forecast_initial_time=0)

        # Helper function to process each GHI dataset (calculates hourly accumulation)
        def get_hourly_ssrd(ds_ghi_processed):
            ssrd_data = ds_ghi_processed['SSRD'] # Dimensions are now (forecast_hour, latitude, longitude)

            # Reconstruct valid_time using forecast_initial_time and forecast_hour
            # forecast_hour is in hours, so convert to timedelta
            valid_time_coord = ds_ghi_processed['forecast_initial_time'].values + pd.to_timedelta(ds_ghi_processed['forecast_hour'].values, unit='h')

            # Calculate hourly accumulation (difference over 'forecast_hour' dimension)
            hourly_ssrd = ssrd_data.diff(dim='forecast_hour')

            # Assign valid_time values (excluding the first one) to a new 'time' coordinate
            hourly_ssrd = hourly_ssrd.assign_coords(forecast_hour=valid_time_coord[1:]) # Temporarily assign to forecast_hour coord
            hourly_ssrd = hourly_ssrd.rename({'forecast_hour': 'time'}) # Rename 'forecast_hour' to 'time'

            return hourly_ssrd

        # Process both GHI parts
        g1_hourly_ssrd = get_hourly_ssrd(ds_g1)
        g2_hourly_ssrd = get_hourly_ssrd(ds_g2)

        # Concatenate the two hourly GHI DataArrays along the 'time' dimension, with compat='override'
        ds_g = xr.concat([g1_hourly_ssrd, g2_hourly_ssrd], dim='time', coords='minimal', compat='override')

        # Select the city coordinates on the analysis data (temperature, u-wind, v-wind)
        t = ds_t.sel(latitude=lat, longitude=adj_lon, method='nearest')
        u = ds_u.sel(latitude=lat, longitude=adj_lon, method='nearest')
        v = ds_v.sel(latitude=lat, longitude=adj_lon, method='nearest')

        # Select the city coordinates on the processed GHI data
        g = ds_g.sel(latitude=lat, longitude=adj_lon, method='nearest')

        # Create the combined DataFrame
        # Note: GHI (SSRD) is in J/m2, we divide by 3600 to get average W/m2 for that hour
        df = pd.DataFrame({
            'Time': t.time.values,
            'Temp_C': t['VAR_2T'].values - 273.15, # Corrected variable name
            'Wind_ms': np.sqrt(u['VAR_10U']**2 + v['VAR_10V']**2), # Corrected variable names
            # g is now a DataArray after sel. Apply reindex_like on it.
            # Use fill_value=0 for missing GHI values (e.g., before 06:00 on Jan 1)
            'GHI_Wm2': g.reindex_like(t, method='nearest', fill_value=0).values / 3600.0
        })

        df.to_csv(f"{city_name.lower()}_s3_2024.csv", index=False)
        print(f"✅ Created {city_name.lower()}_s3_2024.csv directly from NCAR S3.")
        return df.head()

    except Exception as e:
        print(f"❌ Error extracting {city_name}: {e}")

# Define cities
cities = {
    "Turku": [60.45, 22.26],
    "Stockholm": [59.33, 18.07],
    "Oslo": [59.91, 10.75],
    "Copenhagen": [55.68, 12.57]
}

# Loop through cities and extract data
for city_name, coords in cities.items():
    lat, lon = coords
    extract_all_vars(lat, lon, city_name)

✅ Created turku_s3_2024.csv directly from NCAR S3.
✅ Created stockholm_s3_2024.csv directly from NCAR S3.
✅ Created oslo_s3_2024.csv directly from NCAR S3.
✅ Created copenhagen_s3_2024.csv directly from NCAR S3.


## Final Task

### Subtask:
Confirm that the weather data for all cities for January 2024 has been successfully downloaded from S3, processed, and saved to CSV files without any further errors. If successful, describe the content of the generated CSV files.


## Summary:

### Q&A
Yes, the weather data for all specified cities (Turku, Stockholm, Oslo, and Copenhagen) for January 2024 has been successfully downloaded from S3, processed, and saved to individual CSV files. The final execution of the `extract_all_vars` function completed without errors.

The generated CSV files (e.g., `turku_s3_2024.csv`) contain the following columns for each city for January 2024, with hourly resolution:
*   `Time`: Timestamp of the data point.
*   `Temp_C`: Air temperature at 2 meters, converted from Kelvin to Celsius.
*   `Wind_ms`: Wind speed at 10 meters, calculated from the u-component and v-component of wind.
*   `GHI_Wm2`: Global Horizontal Irradiance, representing hourly accumulated solar radiation, converted from J/m² to W/m².

### Data Analysis Key Findings
*   The variable names for temperature, u-component of wind, and v-component of wind were successfully corrected to `VAR_2T`, `VAR_10U`, and `VAR_10V`, respectively.
*   Processing of Global Horizontal Irradiance (GHI) data presented several challenges, primarily due to the accumulated nature of forecast data and `xarray`'s handling of time dimensions:
    *   Initial attempts to concatenate GHI datasets failed due to conflicting dimension sizes.
    *   Correct extraction of GHI required selecting the appropriate `forecast_initial_time` (using `isel(forecast_initial_time=0)`).
    *   The `valid_time` coordinate for GHI needed to be reconstructed from `forecast_initial_time` and `forecast_hour` before calculating hourly differences.
    *   Concatenating the two GHI parts required explicit `coords='minimal'` and `compat='override'` parameters in `xr.concat` to resolve conflicts in non-concatenated dimensions and suppress warnings, ensuring proper merging of the time series.
*   All four CSV files (`turku_s3_2024.csv`, `stockholm_s3_2024.csv`, `oslo_s3_2024.csv`, `copenhagen_s3_2024.csv`) were successfully generated for January 2024, each containing hourly weather data.

### Insights or Next Steps
*   The detailed debugging required for GHI data highlights the complexity of working with forecast-based accumulated variables and the importance of understanding `xarray`'s dimension and coordinate handling (`isel`, `assign_coords`, `concat` parameters).
*   For future data extraction tasks involving similarly structured forecast data, consider creating a generalized utility function to encapsulate the complex GHI processing logic, enhancing reusability and reducing potential errors.
